In [7]:
"""
目標
1. 設定一個threshold值(15%)，從台股中找出(股價20T均值/現值)>=(1+threshold)的標的(股票代碼+名稱)，供下單參考
2. 找出的標的再拿去Taiwan_Stock_Analysis做檢查，決定是否下單

研究(來源為yahoo股市)
有資料的部分(MA20的值)

裕隆
https://s.yimg.com/nb/tw_stock_frontend/scripts/TaChart/tachart.14b50e7f13.html?sid=2201&w=660
玉山金
https://s.yimg.com/nb/tw_stock_frontend/scripts/TaChart/tachart.14b50e7f13.html?sid=2884&w=660
台積電
https://s.yimg.com/nb/tw_stock_frontend/scripts/TaChart/tachart.14b50e7f13.html?sid=2330&w=660


觀察:
只有sid的部分會不同 -> 填上個股代碼即可


研究
裕隆(2201)
3/1: https://tw.quote.finance.yahoo.net/quote/q?type=ta&perd=d&mkt=10&sym=2201&v=1&callback=jQuery1113011211715523058885_1677651487486&_=1677651487487
3/2: https://tw.quote.finance.yahoo.net/quote/q?type=ta&perd=d&mkt=10&sym=2201&v=1&callback=jQuery111308492919127262011_1677723029059&_=1677723029060
台積電(2330)
https://tw.quote.finance.yahoo.net/quote/q?type=ta&perd=d&mkt=10&sym=2330&v=1&callback=jQuery111309744274145875191_1677654204317&_=1677654204318
玉山金(2884)
https://tw.quote.finance.yahoo.net/quote/q?type=ta&perd=d&mkt=10&sym=2884&v=1&callback=jQuery1113012607446060399075_1677654447419&_=1677654447420

觀察:
除了sym不一樣，後面的callback=jQueryXXX也不同 -> 
  1.找出規律 -> 得到的資料需自行計算 -> 不建議!
  2.從其他管道找資料 -> 從Goodinfo!台灣股市交易網 >> K線比較圖 中取得原始資料，自行計算
同一支股票，不同天去看url，結果也不同!
模仿ajax非同步的方式來取得所需的資料? -> try try 看


研究(以裕隆為例；由後往前回溯)
1.https://s.yimg.com/nb/tw_stock_frontend/scripts/TaChart/tachart.14b50e7f13.html?sid=2201&w=660 (有資料的地方)
  ↑↑
2.https://s.yimg.com/ud/others/stock/jquery.1.11.3.min.js
  ↑↑
3.https://s.yimg.com/nb/tw_stock_frontend/scripts/TaChart/tachart.14b50e7f13.html?sid=2201&w=660


觀察
數據的來源url有規律(僅sid不同)，但卡在需要執行jquery.1.11.3.min.js檔才會有數據出來



研究(來源違Goodinfo!)
裕隆
https://goodinfo.tw/tw/ShowK_ChartCompare.asp?STOCK_ID=2201&STOCK_ID1=%E5%8A%A0%E6%AC%8A%E6%8C%87%E6%95%B8&CHT_CAT2=DATE
玉山金
https://goodinfo.tw/tw/ShowK_ChartCompare.asp?STOCK_ID=2884&STOCK_ID1=%E5%8A%A0%E6%AC%8A%E6%8C%87%E6%95%B8&CHT_CAT2=DATE
台積電
https://goodinfo.tw/tw/ShowK_ChartCompare.asp?STOCK_ID=2330&STOCK_ID1=%E5%8A%A0%E6%AC%8A%E6%8C%87%E6%95%B8&CHT_CAT2=DATE


觀察:
只有STOCK_ID=這一項不同，其他都一樣 -> 從這裡來算20MA值! -> 考慮到server可能會擋過於頻繁的request，可能得找其他source?

"""

#使用到的packages
import requests
from bs4 import BeautifulSoup as BS
import time
import re
import random

#selenium套件可用來做為自動化測試套件
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By



#from selenium.webdriver.chrome.options import Options

#變數
collected_data = {} #紀錄個股數據
collected_stocks = {} #紀錄符合條件的個股

#閥值->超過這個比例才紀錄下來
#threshold = 0.15  

print('找出「MA20/現價」>= (1+threshold) 的股票代碼')
#low_bound = input('請輸入欲分析的股票代碼下限值: ')
#up_bound = input('請輸入欲分析的股票代碼上限值: ')
threshold = input('請輸入threshold值(0以上): ')


#取得台股代碼一覽表
#參考網頁: https://stock.wespai.com/p/3752
#必要時再打開來執行
"""
url_stock_list = 'https://stock.wespai.com/p/3752'

#user-agent定義，防止爬蟲被擋
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"
headers = {'user-agent': user_agent}

res = requests.get(url_stock_list, headers=headers, verify=False)
res.encoding = 'utf-8'


soup = BS(res.text,'html.parser')
#print(soup)
table = soup.find('table', id='example')
tbody = table.find('tbody')
print(tbody)

trs = tbody.find_all('tr')

stock_list = []
for tr in trs:
    code = tr.find('td').text
    stock_list.append(code)

print(stock_list)
"""

#共有1762筆
got_stock_list = ['1101', '1102', '1103', '1104', '1108', '1109', '1110', '1201', '1203', '1210', '1213', '1215', '1216', '1217', '1218', '1219', '1220', '1225', '1227', '1229', '1231', '1232', '1233', '1234', '1235', '1236', '1256', '1301', '1303', '1304', '1305', '1307', '1308', '1309', '1310', '1312', '1313', '1314', '1315', '1316', '1319', '1321', '1323', '1324', '1325', '1326', '1337', '1338', '1339', '1340', '1341', '1342', '1402', '1409', '1410', '1413', '1414', '1416', '1417', '1418', '1419', '1423', '1432', '1434', '1436', '1437', '1438', '1439', '1440', '1441', '1442', '1443', '1444', '1445', '1446', '1447', '1449', '1451', '1452', '1453', '1454', '1455', '1456', '1457', '1459', '1460', '1463', '1464', '1465', '1466', '1467', '1468', '1470', '1471', '1472', '1473', '1474', '1475', '1476', '1477', '1503', '1504', '1506', '1513', '1514', '1515', '1516', '1517', '1519', '1521', '1522', '1524', '1525', '1526', '1527', '1528', '1529', '1530', '1531', '1532', '1533', '1535', '1536', '1537', '1538', '1539', '1540', '1541', '1558', '1560', '1568', '1582', '1583', '1587', '1589', '1590', '1597', '1598', '1603', '1604', '1605', '1608', '1609', '1611', '1612', '1614', '1615', '1616', '1617', '1618', '1626', '1701', '1702', '1707', '1708', '1709', '1710', '1711', '1712', '1713', '1714', '1717', '1718', '1720', '1721', '1722', '1723', '1725', '1726', '1727', '1730', '1731', '1732', '1733', '1734', '1735', '1736', '1737', '1752', '1760', '1762', '1773', '1776', '1783', '1786', '1789', '1795', '1802', '1805', '1806', '1808', '1809', '1810', '1817', '1903', '1904', '1905', '1906', '1907', '1909', '2002', '2006', '2007', '2008', '2009', '2010', '2012', '2013', '2014', '2015', '2017', '2020', '2022', '2023', '2024', '2025', '2027', '2028', '2029', '2030', '2031', '2032', '2033', '2034', '2038', '2049', '2059', '2062', '2069', '2101', '2102', '2103', '2104', '2105', '2106', '2107', '2108', '2109', '2114', '2115', '2201', '2204', '2206', '2207', '2208', '2211', '2227', '2228', '2231', '2233', '2236', '2239', '2241', '2243', '2247', '2250', '2301', '2302', '2303', '2305', '2308', '2312', '2313', '2314', '2316', '2317', '2321', '2323', '2324', '2327', '2328', '2329', '2330', '2331', '2332', '2337', '2338', '2340', '2342', '2344', '2345', '2347', '2348', '2349', '2351', '2352', '2353', '2354', '2355', '2356', '2357', '2358', '2359', '2360', '2362', '2363', '2364', '2365', '2367', '2368', '2369', '2371', '2373', '2374', '2375', '2376', '2377', '2379', '2380', '2382', '2383', '2385', '2387', '2388', '2390', '2392', '2393', '2395', '2397', '2399', '2401', '2402', '2404', '2405', '2406', '2408', '2409', '2412', '2413', '2414', '2415', '2417', '2419', '2420', '2421', '2423', '2424', '2425', '2426', '2427', '2428', '2429', '2430', '2431', '2433', '2434', '2436', '2438', '2439', '2440', '2441', '2442', '2443', '2444', '2449', '2450', '2451', '2453', '2454', '2455', '2457', '2458', '2459', '2460', '2461', '2462', '2464', '2465', '2466', '2467', '2468', '2471', '2472', '2474', '2476', '2477', '2478', '2480', '2481', '2482', '2483', '2484', '2485', '2486', '2488', '2489', '2491', '2492', '2493', '2495', '2496', '2497', '2498', '2501', '2504', '2505', '2506', '2509', '2511', '2514', '2515', '2516', '2520', '2524', '2527', '2528', '2530', '2534', '2535', '2536', '2537', '2538', '2539', '2540', '2542', '2543', '2545', '2546', '2547', '2548', '2597', '2601', '2603', '2605', '2606', '2607', '2608', '2609', '2610', '2611', '2612', '2613', '2614', '2615', '2616', '2617', '2618', '2630', '2633', '2634', '2636', '2637', '2642', '2701', '2702', '2704', '2705', '2706', '2707', '2712', '2722', '2723', '2727', '2731', '2739', '2748', '2753', '2801', '2809', '2812', '2816', '2820', '2832', '2834', '2836', '2838', '2845', '2849', '2850', '2851', '2852', '2855', '2867', '2880', '2881', '2882', '2883', '2884', '2885', '2886', '2887', '2888', '2889', '2890', '2891', '2892', '2897', '2901', '2903', '2904', '2905', '2906', '2908', '2910', '2911', '2912', '2913', '2915', '2923', '2929', '2939', '2945', '3002', '3003', '3004', '3005', '3006', '3010', '3011', '3013', '3014', '3015', '3016', '3017', '3019', '3021', '3022', '3023', '3024', '3025', '3026', '3027', '3028', '3029', '3030', '3031', '3032', '3033', '3034', '3035', '3036', '3037', '3038', '3040', '3041', '3042', '3043', '3044', '3045', '3046', '3047', '3048', '3049', '3050', '3051', '3052', '3054', '3055', '3056', '3057', '3058', '3059', '3060', '3062', '3090', '3092', '3094', '3130', '3138', '3149', '3164', '3167', '3189', '3209', '3229', '3231', '3257', '3266', '3296', '3305', '3308', '3311', '3312', '3321', '3338', '3346', '3356', '3376', '3380', '3406', '3413', '3416', '3419', '3432', '3437', '3443', '3447', '3450', '3454', '3481', '3494', '3501', '3504', '3515', '3518', '3528', '3530', '3532', '3533', '3535', '3536', '3543', '3545', '3550', '3557', '3563', '3576', '3583', '3588', '3591', '3592', '3593', '3596', '3605', '3607', '3617', '3622', '3645', '3652', '3653', '3661', '3665', '3669', '3673', '3679', '3682', '3686', '3694', '3701', '3702', '3703', '3704', '3705', '3706', '3708', '3711', '3714', '4104', '4106', '4108', '4119', '4133', '4137', '4142', '4148', '4155', '4164', '4190', '4306', '4426', '4438', '4439', '4440', '4526', '4532', '4536', '4540', '4545', '4551', '4552', '4555', '4557', '4560', '4562', '4564', '4566', '4571', '4572', '4576', '4581', '4583', '4720', '4722', '4737', '4739', '4746', '4755', '4763', '4764', '4766', '4770', '4807', '4904', '4906', '4912', '4915', '4916', '4919', '4927', '4930', '4934', '4935', '4938', '4942', '4943', '4952', '4956', '4958', '4960', '4961', '4967', '4968', '4976', '4977', '4989', '4994', '4999', '5007', '5203', '5215', '5222', '5225', '5234', '5243', '5244', '5258', '5269', '5283', '5284', '5285', '5288', '5306', '5388', '5434', '5469', '5471', '5484', '5515', '5519', '5521', '5522', '5525', '5531', '5533', '5534', '5538', '5546', '5607', '5608', '5706', '5871', '5876', '5880', '5906', '5907', '6005', '6024', '6108', '6112', '6115', '6116', '6117', '6120', '6128', '6133', '6136', '6139', '6141', '6142', '6152', '6153', '6155', '6164', '6165', '6166', '6168', '6176', '6177', '6183', '6184', '6189', '6191', '6192', '6196', '6197', '6201', '6202', '6205', '6206', '6209', '6213', '6214', '6215', '6216', '6224', '6225', '6226', '6230', '6235', '6239', '6243', '6257', '6269', '6271', '6277', '6278', '6281', '6282', '6283', '6285', '6288', '6405', '6412', '6414', '6415', '6416', '6426', '6431', '6438', '6442', '6443', '6449', '6451', '6456', '6464', '6477', '6491', '6504', '6505', '6515', '6525', '6531', '6533', '6550', '6552', '6558', '6573', '6579', '6581', '6582', '6585', '6591', '6592', '6598', '6605', '6625', '6641', '6655', '6666', '6668', '6669', '6670', '6671', '6672', '6674', '6689', '6691', '6695', '6698', '6706', '6715', '6719', '6743', '6754', '6756', '6768', '6770', '6776', '6781', '6782', '6789', '6790', '6796', '6799', '6806', '6807', '6830', '6834', '6835', '8011', '8016', '8021', '8028', '8033', '8039', '8046', '8070', '8072', '8081', '8101', '8103', '8104', '8105', '8110', '8112', '8114', '8131', '8150', '8163', '8201', '8210', '8213', '8215', '8222', '8249', '8261', '8271', '8341', '8367', '8374', '8404', '8411', '8422', '8429', '8438', '8442', '8443', '8454', '8462', '8463', '8464', '8466', '8467', '8473', '8478', '8481', '8482', '8488', '8499', '8926', '8940', '8996', '9105', '911622', '9802', '9902', '9904', '9905', '9906', '9907', '9908', '9910', '9911', '9912', '9914', '9917', '9918', '9919', '9921', '9924', '9925', '9926', '9927', '9928', '9929', '9930', '9931', '9933', '9934', '9935', '9937', '9938', '9939', '9940', '9941', '9942', '9943', '9944', '9945', '9946', '9955', '9958', '1240', '1258', '1259', '1264', '1268', '1336', '1565', '1569', '1570', '1580', '1584', '1586', '1591', '1593', '1595', '1599', '1742', '1777', '1781', '1784', '1785', '1788', '1796', '1799', '1813', '1815', '2035', '2061', '2063', '2064', '2065', '2066', '2067', '2070', '2221', '2230', '2235', '2596', '2640', '2641', '2643', '2718', '2719', '2724', '2726', '2729', '2732', '2734', '2736', '2740', '2743', '2745', '2752', '2754', '2755', '2756', '2916', '2924', '2926', '2937', '3064', '3066', '3067', '3071', '3078', '3081', '3083', '3085', '3086', '3088', '3089', '3093', '3095', '3105', '3114', '3115', '3118', '3122', '3128', '3131', '3141', '3147', '3152', '3162', '3163', '3169', '3171', '3176', '3178', '3188', '3191', '3202', '3205', '3206', '3207', '3211', '3213', '3217', '3218', '3219', '3221', '3224', '3226', '3227', '3228', '3230', '3232', '3234', '3236', '3252', '3259', '3260', '3264', '3265', '3268', '3272', '3276', '3284', '3285', '3287', '3288', '3289', '3290', '3293', '3294', '3297', '3303', '3306', '3310', '3313', '3317', '3322', '3323', '3324', '3325', '3332', '3339', '3349', '3354', '3357', '3360', '3362', '3363', '3372', '3373', '3374', '3379', '3388', '3390', '3402', '3426', '3430', '3434', '3438', '3441', '3444', '3455', '3465', '3466', '3479', '3483', '3484', '3489', '3490', '3491', '3492', '3498', '3499', '3508', '3511', '3512', '3516', '3520', '3521', '3522', '3523', '3526', '3527', '3531', '3537', '3540', '3541', '3546', '3548', '3551', '3552', '3555', '3556', '3558', '3564', '3567', '3570', '3577', '3580', '3581', '3587', '3594', '3597', '3609', '3611', '3615', '3623', '3624', '3625', '3628', '3629', '3630', '3631', '3632', '3646', '3663', '3664', '3666', '3672', '3675', '3680', '3684', '3685', '3687', '3689', '3691', '3693', '3707', '3709', '3710', '3713', '4102', '4105', '4107', '4109', '4111', '4113', '4114', '4116', '4120', '4121', '4123', '4126', '4127', '4128', '4129', '4130', '4131', '4138', '4139', '4147', '4153', '4154', '4157', '4160', '4161', '4162', '4163', '4167', '4168', '4171', '4173', '4174', '4175', '4183', '4188', '4192', '4198', '4205', '4207', '4303', '4304', '4305', '4401', '4402', '4406', '4413', '4416', '4417', '4419', '4420', '4430', '4432', '4433', '4502', '4503', '4506', '4510', '4513', '4523', '4527', '4528', '4529', '4530', '4533', '4534', '4535', '4538', '4541', '4542', '4543', '4549', '4550', '4554', '4556', '4558', '4561', '4563', '4568', '4577', '4580', '4584', '4609', '4702', '4706', '4707', '4711', '4712', '4714', '4716', '4721', '4726', '4728', '4729', '4735', '4736', '4741', '4743', '4744', '4745', '4747', '4754', '4760', '4767', '4768', '4804', '4806', '4903', '4905', '4907', '4908', '4909', '4911', '4923', '4924', '4931', '4933', '4939', '4944', '4945', '4946', '4950', '4951', '4953', '4966', '4971', '4972', '4973', '4974', '4979', '4987', '4991', '4995', '5009', '5011', '5013', '5014', '5015', '5016', '5201', '5202', '5205', '5206', '5209', '5210', '5211', '5212', '5213', '5220', '5223', '5227', '5228', '5230', '5236', '5245', '5251', '5263', '5272', '5276', '5278', '5281', '5287', '5289', '5291', '5299', '5301', '5302', '5309', '5310', '5312', '5314', '5315', '5321', '5324', '5328', '5340', '5344', '5345', '5347', '5348', '5351', '5353', '5355', '5356', '5364', '5371', '5381', '5383', '5386', '5392', '5398', '5403', '5410', '5425', '5426', '5432', '5438', '5439', '5443', '5450', '5452', '5455', '5457', '5460', '5464', '5465', '5468', '5474', '5475', '5478', '5481', '5483', '5487', '5488', '5489', '5490', '5493', '5498', '5508', '5511', '5512', '5514', '5516', '5520', '5523', '5529', '5530', '5536', '5543', '5601', '5603', '5604', '5609', '5701', '5703', '5704', '5864', '5878', '5902', '5903', '5904', '5905', '6015', '6016', '6020', '6021', '6023', '6026', '6101', '6104', '6109', '6111', '6113', '6114', '6118', '6121', '6122', '6123', '6124', '6125', '6126', '6127', '6129', '6130', '6134', '6138', '6140', '6143', '6144', '6146', '6147', '6148', '6150', '6151', '6154', '6156', '6158', '6160', '6161', '6163', '6167', '6169', '6170', '6171', '6173', '6174', '6175', '6179', '6180', '6182', '6185', '6186', '6187', '6188', '6190', '6194', '6195', '6198', '6199', '6203', '6204', '6207', '6208', '6210', '6212', '6217', '6218', '6219', '6220', '6221', '6222', '6223', '6227', '6228', '6229', '6231', '6233', '6234', '6236', '6237', '6240', '6241', '6242', '6244', '6245', '6246', '6248', '6259', '6261', '6263', '6264', '6265', '6266', '6270', '6274', '6275', '6276', '6279', '6284', '6287', '6290', '6291', '6292', '6294', '6404', '6411', '6417', '6418', '6419', '6425', '6432', '6435', '6441', '6446', '6457', '6461', '6462', '6465', '6469', '6470', '6472', '6482', '6485', '6486', '6488', '6492', '6494', '6496', '6499', '6506', '6508', '6509', '6510', '6512', '6514', '6516', '6517', '6523', '6527', '6530', '6532', '6535', '6538', '6542', '6546', '6547', '6548', '6556', '6560', '6561', '6568', '6569', '6570', '6574', '6576', '6577', '6578', '6588', '6589', '6590', '6593', '6594', '6596', '6603', '6609', '6612', '6613', '6615', '6616', '6624', '6629', '6640', '6642', '6643', '6649', '6651', '6654', '6661', '6662', '6667', '6679', '6680', '6683', '6684', '6690', '6693', '6697', '6703', '6708', '6712', '6716', '6721', '6727', '6728', '6732', '6733', '6735', '6741', '6747', '6751', '6752', '6761', '6762', '6763', '6767', '6788', '6791', '6803', '6804', '6823', '6829', '6841', '6843', '6855', '6859', '6865', '6874', '7402', '7556', '8024', '8027', '8032', '8034', '8038', '8040', '8042', '8043', '8044', '8047', '8048', '8049', '8050', '8054', '8059', '8064', '8066', '8067', '8068', '8069', '8071', '8074', '8076', '8077', '8083', '8084', '8085', '8086', '8087', '8088', '8089', '8091', '8092', '8093', '8096', '8097', '8099', '8107', '8109', '8111', '8121', '8147', '8155', '8171', '8176', '8182', '8183', '8227', '8234', '8240', '8255', '8277', '8279', '8284', '8289', '8291', '8299', '8342', '8349', '8354', '8358', '8383', '8390', '8401', '8403', '8409', '8410', '8415', '8416', '8418', '8420', '8421', '8423', '8424', '8426', '8431', '8432', '8433', '8435', '8436', '8437', '8440', '8444', '8446', '8450', '8455', '8472', '8476', '8477', '8489', '8905', '8906', '8908', '8916', '8917', '8921', '8923', '8924', '8927', '8928', '8929', '8930', '8931', '8932', '8933', '8935', '8936', '8937', '8938', '8941', '8942', '9949', '9950', '9951', '9960', '9962','911622']

#記錄垃圾股票代碼，直接跳過不要再分析
garbage_stock_list = ['1325','1316','1309','1312','1456','1455','1459','1440','1515','1452','1612','1727','2007','1773',
                     '1903','2010','2022','2023','1905','1721','2030','2108','2069','2314','2038','2241','2250','2390',
                     '2369','2358','2401','2415','2348','2436','2465','2458','2440','2441','2426','2455','2477','2491',
                     '2486','2489','2492','2615','2636','2609','2611','2537','2614','2527','2540','2867','2851','2915',
                     '2882','2905','2855','3006','2441','2642','3016','3037','3092','3149','3041','3167','3054','3257',
                     '3305','3321','3588','3545','3532','3536','3645','3530','3605','3682','3714','4564','4739','4576',
                     '4571','4943','4968','5471','4927','5269','4989','5546','5608','4956','6202','6141','6164','6224',
                     '6415','6582','6706','6573','6504','6756','6552','6431','8104','8404','9945','9921','9919','1815',
                     '1785','2067','3228','3122','3105','3141','3086','3152','3219','3176','3230','3221','3484','3339',
                     '3373','3374','3294','3372','3527','3465','3691','3625','3709','3555','3687','3685','3629','4123',
                     '3672','4171','4303','4154','4174','4139','4192','4554','4944','4806','4945','4760','4721','4735',
                     '4946','4966','4976','4971','4939','4714','5299','5351','5302','5347','5245','5344','5202','5278',
                     '5475','5603','5487','5529','5468','5457','6026','6015','5604','5474','6104','6021','6111','6198',
                     '6187','6150','6158','6173','6127','6204','6182','6138','6219','6229','6457','6241','6494','6237',
                     '6485','6411','6435','6488','6462','6265','6492','6259','6499','6547','6548','6560','6613','6716',
                     '6568','6651','6683','6654','6616','6588','6642','8044','8086','8054','8027','8477','8121','8444',
                     '8450','8489','8182','5215''5306','8454','3707','5536','1313','1310','1101','1326','1308','1109',
                     '1314','1305','1338','1517','1466','1475','1460','1476','1467','1516','1474','1536','1447','1558',
                     '1541','1608','1711','1776','1732','1722','1806','1802','2014','1909','2227','2020','1907','1906',
                     '2104','2103','1904','2012','2028','2032','2029','2114','2101','2115','2102','2323','2838','2340',
                     '2357','2380','2393','2363','2374','2481','2478','2434','2505','2460','2520','2516','2524','2613',
                     '2612','2601','2816','2610','2607','2637','2883','2881','2884','2888','3003','2891','3048','3049',
                     '2929','3515','3060','3543','3593','4439','4555','5306','4807','5531','5538','4935','6271','6005',
                     '6108','6225','6671','6405','6672','8463','8021','8150','8481','8110','8341','8466','8499','9938',
                     '1586','9946','1599','3188','3317','3227','3259','3354','3226','3631','4712','4401','4716','4530',
                     '4711','4561','4528','4950','5223','5230','5016','4991','5432','5398','5321','5498','5543','5356',
                     '5516','6171','5905','6016']

#紀錄目標股票代碼，也不用再分析
target_stock_list = ['3189','4912','6213','6781','8046','9914','3093','3169','3081','6274','6741','8255','1227','1725',
                    '2059','2383','3450','4133','5215']

#篩選過後的list
garbage_filtered_stock_list = list(filter(lambda stock : all(stock != garbage_id for garbage_id in garbage_stock_list), got_stock_list))
filtered_stock_list = list(filter(lambda stock : all(stock != target_id for target_id in target_stock_list), garbage_filtered_stock_list))

#需要將list每500個切成一組，因為server的限制!
#server另一個限制: 可能是連續N分鐘一直請求會被斷掉?
#解法: 將list切好，再去run
#分割list(一次50個差不多)
#等到篩出來的collected_stocks沒有東西之後，就可以更改range選取的範圍
split_list = []
print(len(filtered_stock_list))
for i in range(1100,1150):
    if(filtered_stock_list[i] is not None):
        split_list.append(filtered_stock_list[i])

print(split_list)
print(len(split_list))

#test_list = ['1101', '1102', '1103', '1104', '1108', '1109', '1110']

#測試 -> 用edge來跑 -> 可以了?!?! -> 版本號要一模一樣!
print('開始連線')
rnd_num1 = random.random() + 1
browser = webdriver.Edge(r'msedgedriver.exe')
time.sleep(rnd_num1)  #單位:sec

#計數用的變數
counter = 0

#url相關變數
url_prev = 'http://s.yimg.com/nb/tw_stock_frontend/scripts/TaChart/tachart.14b50e7f13.html?sid='
url_post = '&w=660'

#skip 1 error!

#用迴圈來爬各個股票的資料
for stock_code in split_list:
    
    
        
    url = url_prev + str(stock_code) + url_post

    browser.get(url)  #此為日線/成交量選項->需改成月線!
    browser.refresh()

    rnd_num2 = random.random() + 2
    time.sleep(rnd_num2)  #單位:sec


    #選擇下拉式選單中的「月線」
    select = Select(browser.find_element(By.ID, 'TAChartPeriod'))
    select.select_by_value('m')
    time.sleep(2)  #單位:sec

    #爬取網頁內容
    soup = BS(browser.page_source,'html.parser')
    #print(soup.text)


    #若有爬到東西，就從中撈資料；else跳過
    if(not re.search('.*No\sData$',soup.text)):
        #計算「MA20/現價」是否>1+threshold值，若有的化就記錄下來
        tachartlabel1 = soup.find('div', id='TAChartLabel1')
        tacharttitle = soup.find('label', id='TaChartTitle')

        #print(tachartlabel1)
        #print(tachartlabel1.text)
        #print(tachartlabel1.text[0])
        #print(type(tachartlabel1))

        
        #考慮到有些台股代碼不是4位數 -> 也有6位數的
        name = ''
        if(re.search('.+\(\d{4}\)',tacharttitle.text) is not None):
            name = re.search('.+\(\d{4}\)',tacharttitle.text).group(0)
        elif(re.search('.+\(\d{6}\)',tacharttitle.text) is not None):
            name = re.search('.+\(\d{6}\)',tacharttitle.text).group(0)
        else:
            continue
            
       
        date = re.search('\d{4}/\d{2}/\d{2}',tachartlabel1.text).group(0)
        current_price = float(re.search('收(\d+\.?\d*)\s',tachartlabel1.text).group(1))
        
        #考慮到有些股票沒有月線的MA20 -> 直接跳過
        MA20 = 0
        if(re.search('MA20\s(\d+\.\d+)\s',tachartlabel1.text) is not None):
            MA20 = float(re.search('MA20\s(\d+\.\d+)\s',tachartlabel1.text).group(1))
        else:
            continue
        
        ratio = MA20 / current_price

        collected_data['name'] = name
        collected_data['date'] = date
        collected_data['current_price'] = current_price
        collected_data['MA20'] = MA20
        collected_data['ratio'] = ratio
        print(collected_data)


        if(ratio >= (1 + float(threshold))):
            temp_dict = collected_data.copy()  #需要做copy, 不然collected_data會對應到同一個記憶體位址!
            collected_stocks[name] = temp_dict  
        
        
        counter = counter + 1
        #每10個做一次輸出 -> 一個counter來計數
        if(counter == 10):
            counter = 0
            browser.close()
            print('連線結束')
            counter = 0
            print('批次輸出:')
            print(collected_stocks)

            print('開始連線')
            rnd_num1 = random.random() + 1
            browser = webdriver.Edge(r'msedgedriver.exe')
            time.sleep(rnd_num1)  #單位:sec
            #collected_data.clear()
    else:
        continue
    
        
    
    
 
browser.close()
print('連線結束')

print('ratio超過threshold值得股票:')
print(collected_stocks)

#將結果按照ratio值由大到小排序
sorted_collected_stocks = sorted(collected_stocks.items(), key = lambda x:x[1]['ratio'], reverse = True)
print('排序過後:')
print(sorted_collected_stocks)



"""

#options = webdriver.ChromeOptions()
options = Options()
options.add_argument('--no-sandbox')
options.add_argument('--headless')
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36")
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-notifications')
options.binary_location = './chromedriver.exe'

print('開使連線')
chrome = webdriver.Chrome('chromedriver', options=options)
browser = chrome.get("http://s.yimg.com/nb/tw_stock_frontend/scripts/TaChart/tachart.14b50e7f13.html?sid=2330&w=660")
myData = browser.find_element_by_id('#TAChartLabel1')
#browser = chrome.get('https://ithelp.ithome.com.tw/articles/10230026')  #測試用
#browser = chrome.get('http://www.google.com') #測試用
browser.close()
print('關閉連線')
print(myData)
#print('Hello')

#參考: https://medium.com/@bob800530/selenium-1-%E9%96%8B%E5%95%9Fchrome%E7%80%8F%E8%A6%BD%E5%99%A8-21448980dff9
#參考: https://blog.csdn.net/ouyanggengcheng/article/details/90476553





#爬蟲獲取Goodinfo!K線比較圖中的資料
stock_code = input('請輸入股票代碼: ')
#stock_code = str(2330)  #測試用，到時候要註記掉

url_prev = 'https://goodinfo.tw/tw/ShowK_ChartCompare.asp?STOCK_ID='
url_post = '&STOCK_ID1=%E5%8A%A0%E6%AC%8A%E6%8C%87%E6%95%B8&CHT_CAT2=DATE'
url = url_prev + stock_code + url_post

url_test_prev = 'https://s.yimg.com/nb/tw_stock_frontend/scripts/TaChart/tachart.14b50e7f13.html?sid='
url_test_post = '&w=660'
url_test = url_test_prev + stock_code + url_test_post


#print(url) OK

#user-agent定義，防止爬蟲被擋
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"
headers = {'user-agent': user_agent}


#測試 -> 從yahoo股市爬蟲，是否server會擋過於頻繁的requests
"""


找出「MA20/現價」>= (1+threshold) 的股票代碼
請輸入threshold值(0以上): 0.1
1379
['5904', '6020', '6023', '6101', '6109', '6113', '6114', '6118', '6121', '6122', '6123', '6124', '6125', '6126', '6129', '6130', '6134', '6140', '6143', '6144', '6146', '6147', '6148', '6151', '6154', '6156', '6160', '6161', '6163', '6167', '6169', '6170', '6174', '6175', '6179', '6180', '6185', '6186', '6188', '6190', '6194', '6195', '6199', '6203', '6207', '6208', '6210', '6212', '6217', '6218']
50
開始連線


C:\Users\mi83\AppData\Local\Temp\ipykernel_20144\3019098709.py:183: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Edge(r'msedgedriver.exe')


{'name': '寶雅(5904)', 'date': '2023/03/29', 'current_price': 564.0, 'MA20': 437.48, 'ratio': 0.7756737588652483}
{'name': '大展證(6020)', 'date': '2023/03/29', 'current_price': 18.8, 'MA20': 20.17, 'ratio': 1.072872340425532}
{'name': '元大期(6023)', 'date': '2023/03/29', 'current_price': 50.1, 'MA20': 48.84, 'ratio': 0.9748502994011976}
{'name': '寬魚國際(6101)', 'date': '2023/03/29', 'current_price': 46.45, 'MA20': 23.73, 'ratio': 0.5108719052744887}
{'name': '亞元(6109)', 'date': '2023/03/29', 'current_price': 18.7, 'MA20': 12.65, 'ratio': 0.6764705882352942}
{'name': '亞矽(6113)', 'date': '2023/03/29', 'current_price': 17.8, 'MA20': 16.66, 'ratio': 0.9359550561797753}
{'name': '久威(6114)', 'date': '2023/03/29', 'current_price': 23.75, 'MA20': 23.54, 'ratio': 0.991157894736842}
{'name': '建達(6118)', 'date': '2023/03/29', 'current_price': 35.4, 'MA20': 31.28, 'ratio': 0.8836158192090396}
{'name': '新普(6121)', 'date': '2023/03/29', 'current_price': 311.5, 'MA20': 295.35, 'ratio': 0.9481540930979134}
{'

C:\Users\mi83\AppData\Local\Temp\ipykernel_20144\3019098709.py:278: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Edge(r'msedgedriver.exe')


{'name': '上奇(6123)', 'date': '2023/03/29', 'current_price': 67.9, 'MA20': 68.57, 'ratio': 1.0098674521354931}
{'name': '業強(6124)', 'date': '2023/03/29', 'current_price': 27.15, 'MA20': 25.98, 'ratio': 0.9569060773480663}
{'name': '廣運(6125)', 'date': '2023/03/29', 'current_price': 28.8, 'MA20': 27.26, 'ratio': 0.9465277777777779}
{'name': '信音(6126)', 'date': '2023/03/29', 'current_price': 22.2, 'MA20': 24.03, 'ratio': 1.0824324324324326}
{'name': '普誠(6129)', 'date': '2023/03/29', 'current_price': 34.45, 'MA20': 32.67, 'ratio': 0.9483309143686501}
{'name': '星寶國際(6130)', 'date': '2023/03/29', 'current_price': 35.4, 'MA20': 36.31, 'ratio': 1.0257062146892657}
{'name': '萬旭(6134)', 'date': '2023/03/29', 'current_price': 25.0, 'MA20': 21.7, 'ratio': 0.868}
{'name': '訊達(6140)', 'date': '2023/03/29', 'current_price': 20.55, 'MA20': 17.3, 'ratio': 0.8418491484184915}
{'name': '振曜(6143)', 'date': '2023/03/29', 'current_price': 104.5, 'MA20': 61.17, 'ratio': 0.5853588516746412}
{'name': '得利影(6144)

'\n\n#options = webdriver.ChromeOptions()\noptions = Options()\noptions.add_argument(\'--no-sandbox\')\noptions.add_argument(\'--headless\')\noptions.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36")\noptions.add_argument(\'--disable-dev-shm-usage\')\noptions.add_argument(\'--disable-notifications\')\noptions.binary_location = \'./chromedriver.exe\'\n\nprint(\'開使連線\')\nchrome = webdriver.Chrome(\'chromedriver\', options=options)\nbrowser = chrome.get("http://s.yimg.com/nb/tw_stock_frontend/scripts/TaChart/tachart.14b50e7f13.html?sid=2330&w=660")\nmyData = browser.find_element_by_id(\'#TAChartLabel1\')\n#browser = chrome.get(\'https://ithelp.ithome.com.tw/articles/10230026\')  #測試用\n#browser = chrome.get(\'http://www.google.com\') #測試用\nbrowser.close()\nprint(\'關閉連線\')\nprint(myData)\n#print(\'Hello\')\n\n#參考: https://medium.com/@bob800530/selenium-1-%E9%96%8B%E5%95%9Fchrome%E7%80%8F%E8%A6

In [ ]:
#2023/03/06 開520 高524 低517 收521 量21896 漲跌5MA5 517.80 ▲  MA20 526.00 ▼  MA60 495.23 ▲

In [24]:
!pip list

Package                       Version
----------------------------- --------------------
alabaster                     0.7.12
anaconda-client               1.11.0
anaconda-navigator            2.3.1
anaconda-project              0.11.1
anyio                         3.5.0
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arrow                         1.2.2
astroid                       2.11.7
astropy                       5.1
async-generator               1.10
atomicwrites                  1.4.0
attrs                         21.4.0
Automat                       20.2.0
autopep8                      1.6.0
Babel                         2.9.1
backcall                      0.2.0
backports.functools-lru-cache 1.6.4
backports.tempfile            1.0
backports.weakref             1.0.post1
bcrypt                        3.2.0
beautifulsoup4                4.11.1
binaryornot                   0.4.4
bitarray                      2.5.1
bkc

In [11]:
!pip install --index-url=http://pypi.org/simple/ --trusted-host pypi.org selenium

Looking in indexes: http://pypi.org/simple/


ERROR: Could not find a version that satisfies the requirement selenium (from versions: none)
ERROR: No matching distribution found for selenium
